In [1]:
import numpy as np

In [2]:
# osnovni gradijentni spust
def gradient_descent(f, gradient, x0, alpha, eps, iters):
    x = x0

    for i in range(iters):
        x_new = x - alpha * gradient(x)
        if abs(f(x_new) - f(x)) < eps:
            break
        x = x_new
        
    result = {}
    result['converged'] = (i != iters)
    result['num_iters'] = i
    result['x'] = x_new 
    
    return result

In [3]:
def f(x):
    # neka nam x[0] bude x, a x[1] bude y
    return 0.5 * (x[0]**2 + 10 * x[1]**2)

def gradient(x):
    return np.array([x[0], 10 * x[1]])

In [4]:
x0 = np.array([2, 3])
alpha = 0.1
eps = 0.0001
iters = 1000
gradient_descent(f, gradient, x0, alpha, eps, iters)

{'converged': True, 'num_iters': 40, 'x': array([0.02660559, 0.        ])}

In [5]:
# gradijent sa inercijom
def momentum(f,gradient, x0, alpha, beta, eps, iters):
    x = x0
    d = 0   # inercija
    for i in range(iters):
        # beta je parametar koji kaze koliko vaznosti dajemo toj inerciji
        # NOTE: inercija moze da nas povuce i na stranu suprotnu od one na koju pokazuje gradijent
        d = beta * d + alpha * gradient(x)
        x_new = x - d
        if abs(f(x_new) - f(x)) < eps:
            break
        x = x_new
    
    result = {}
    result['converged'] = (i != iters)
    result['num_iters'] = i
    result['x'] = x_new 
    
    return result

In [6]:
beta = 0.9
momentum(f, gradient, x0, alpha, beta, eps, iters)

{'converged': True, 'num_iters': 107, 'x': array([-0.00663911, -0.0099656 ])}

In [7]:
# nesterovljev algoritam
def nesterov(f,gradient, x0, alpha, beta, eps, iters):
    x = x0
    d = 0   
    for i in range(iters):
        d = beta * d + alpha * gradient(x - beta*d)    # samo se ovde razlikuje u odnosu na prosli
        x_new = x - d
        if abs(f(x_new) - f(x)) < eps:
            break
        x = x_new
    
    result = {}
    result['converged'] = (i != iters)
    result['num_iters'] = i
    result['x'] = x_new 
    
    return result

In [8]:
beta = 0.9
nesterov(f, gradient, x0, alpha, beta, eps, iters)

{'converged': True, 'num_iters': 35, 'x': array([0.01251474, 0.        ])}

In [11]:
def adam(f, gradient, x0, alpha, eps, iters, beta1, beta2, delta):
    x = x0
    m = 0   # prvi momenat
    v = 0   # drugi momenat
    for i in range(1, iters + 1):   # na vezbama je izmenio da ovo ne ide od 0 jer je imao problem
                                    # da deli sa nulom u v_hat racunanju, ali ja taj problem nisam imao
        grad = gradient(x)
        m = beta1 * m + (1 - beta1) * grad
        v = beta2 * v + (1 - beta2) * grad**2
        
        # ovo se radi jer momenat od gradijenta nije nepristrasna ocena (?)
        # nisam pratio bas objasnjenje, pominjao je onaj moving average iz ekonomije
        # ideja je da kako brojevi pristizu, tako oni stariji dosli gube na znacaju
        # kada se raspisu prve recimo 3 iteracije za racunanje m, kad se sredi dobice se
        # m_t = (1-beta1)*sum_i=1_do_t(beta1^(t-i) * g_i)
        # i nisam ispratio kako je dobio odatle ovu fomrulu za ocekivanje, sve g_i smo 
        # pretpsotavili da su jednaki g_t i zato imamo i gresku na kraju
        # E(mt) = E(g_t) * (1-beta1)*sum_i=1_do_t(beta1^(t-i) ) + neka_greska
        # ovo:   (1-beta1)*sum_i=1_do_t(beta1^(t-i) )   po nekoj formuli koji nije naveo na casu
        # to je jednako (1-beta1 ^ t)
        # i na krajud a se dobije m_hat, treba m_t da podelimo sa (1-beta1 ^ t)   (nisam razumeo zasto)
        # i to sada vise nije pristrasno ka nuli (?)
        m_hat = m / (1 - beta1**i)
        # slicno i za v sve
        v_hat = v / (1 - beta2**i)
        
        # note: ovde v moze da bude vektor, i onda je ovo deljenje po koordinatama u numpyju
        # delta je neki mali broj koji se dodaje za svaki slucaj da nebi podelili nulom
        x_new = x - alpha * (m_hat / (np.sqrt(v_hat) + delta))
                             
        if abs(f(x_new) - f(x)) < eps:
            break
        
        x = x_new
    
    result = {}
    result['converged'] = (i != iters)
    result['num_iters'] = i
    result['x'] = x_new 
    
    return result

In [12]:
# ljudi koji su smislili adam algoritamprocenili su da su ovo
# dobre polazne vrednosti za beta1 i beta2
beta1 = 0.9
beta2 = 0.999
adam(f, gradient, x0, alpha, eps, iters, beta1, beta2, 1e-7)

{'converged': True, 'num_iters': 81, 'x': array([0.00410105, 0.00184205])}